In [4]:
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


### Deleting Contents in the JSON file

In [5]:
import json

def clear_sentiment_results(file_name):
    file_path = "results/" + file_name
    try:
        # Open the file in write mode, which will clear its contents
        with open(file_path, 'w') as f:
            # Write an empty list to the file
            json.dump([], f)
        print(f"All objects removed from {file_path}. The file now contains an empty list.")
    except Exception as e:
        print(f"An error occurred while clearing the file: {str(e)}")

# Call the function to clear the file
clear_sentiment_results("sentiment_results.json")

All objects removed from results/sentiment_results.json. The file now contains an empty list.


### New Code with tracking of reasonings and updating prompt

In [6]:
import json

def store_result(result, file_name):
    file_path = "results/" + file_name
    try:
        # Read existing results
        with open(file_path, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        # If file doesn't exist, start with an empty list
        results = []
    
    # Append new result
    results.append(result)
    
    # Write updated results back to file
    with open(file_path, 'w') as f:
        json.dump(results, f, indent=2)

In [15]:
from difflib import SequenceMatcher

def is_similar(a, b, threshold=0.7):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio() > threshold

In [7]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI client
client = OpenAI(api_key=api_key)
    
import json
from openai import OpenAI

client = OpenAI()

sentiment_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "sentiment_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "boolean",
                    "description": "True if in favor of the proposition, false if against"
                },
                "core_reason": {
                    "type": "string",
                    "description": "Brief core argument in sentiment analysis"
                },
                "nuance": {
                    "type": "string",
                    "description": "A new nuance for the core reason, if applicable; empty string if not"
                }
            },
            "required": ["sentiment", "core_reason", "nuance"],
            "additionalProperties": False
        }
    }
}

def analyze_opinion(question, opinion, previous_results):
    try:
        core_reasons = list(previous_results.keys())
        
        system_message = f"""Analyze the given opinion on the question: '{question}'.
        Follow these steps:
        1. Determine the sentiment (true if the opinion is answering yes to the question, false if answering no).
        2. Identify a brief core reason for the sentiment. Check if the core reason is similar to any existing reasons: {core_reasons}
        Guidelines for core arguments:
        - Focus on the primary impact or concern
        - Avoid overly specific or narrow categorizations
        -should be less than 7 words
        - If the core argument is very similar to an existing one, use the exact same wording

        3. State the core reason you've identified as: [CORE_REASON]
        4. I will then provide you with existing nuances for that reason, if any.
        5. Based on the existing nuances (if any), provide a new nuance that captures a key aspect of this opinion.
           Always provide a new nuance, even if it's similar to existing ones.
           The nuance should add depth or a specific example to the core reason.

        Opinion: {opinion}

        Provide steps 1-3, then wait for my response before continuing.
        """
        
        initial_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": opinion}
            ],
            temperature=0.2,
            max_tokens=150
        )
        
        initial_result = initial_response.choices[0].message.content
        core_reason = initial_result.split("[CORE_REASON]")[-1].strip()
        
        existing_nuances = previous_results.get(core_reason, {}).get('nuances', [])
        
        final_message = f"""Now that you've identified the core reason as "{core_reason}", 
        here are the existing nuances for this reason, if any: {existing_nuances}

        Using contrastive learning, carefully compare the current opinion to the existing nuances.
        Only provide a new nuance if it captures a key aspect of this opinion that is not already represented in the existing nuances.
        If the current opinion's perspective is already well-represented, return an empty string for the nuance.

        Original opinion: {opinion}

        """

        final_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18", 
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": opinion},
                {"role": "assistant", "content": initial_result},
                {"role": "user", "content": final_message}
            ],
            response_format=sentiment_schema,
            temperature=0.2,
            max_tokens=300
        )
        
        return json.loads(final_response.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [8]:
def update_core_reasons(result, previous_results):
    try:
        core_reason = result['core_reason']
        sentiment = result['sentiment']
        new_nuance = result['nuance']

        if core_reason not in previous_results:
            previous_results[core_reason] = {
                'sentiment': sentiment,
                'nuances': [],
                'count': 1
            }
        else:
            previous_results[core_reason]['count'] += 1

        if new_nuance:
            previous_results[core_reason]['nuances'].append(new_nuance)

        return previous_results
    except Exception as e:
        print(f"An error occurred while updating core reasons: {str(e)}")
        return previous_results

In [9]:
if __name__ == "__main__":
    file_name = "sentiment_results.json"
    question = "Should phones be banned from schools?"
    

    opinions = [
    "Just as more kids began spending more time with their phones, we saw a massive spike in depression and mental illness.",
    "Phones prevent socialization between students during school.",
    "Despite what rules may exist, most students are using their phones during school",
    "Phone usage reduces learning",
    "Having a smartphone with you at all times gives you the ability to instantly communicate with someone else. Students are able to contact parents, guardians or the authorities without much hassle and vice versa.",
    "Smartphones are all about the apps and the amazing things they can do, these apps can be used in a number of creative ways to facilitate their classroom learning experience.",
    "With phones, students can access research, news and videos to enhance their learning.",
    "Smartphones can be utilized for digital harassment in and out of school.",
    "Students could become highly distracted from the many sources of entertainment.",
    "Frequent usage of smartphones has been linked to negative effects on both physical and mental health.",
    "Banning phones would eliminate the problem of cyberbullying during school hours.",
    "The presence of phones in school creates inequality between students who can and cannot afford them.",
    "Phones are essential for students with certain medical conditions to monitor their health.",
    "The use of phones in schools prepares students for the technology-driven workforce they'll enter.",
    "Phones can be used to cheat on tests and assignments, compromising academic integrity.",
    "Banning phones would make it harder for students to coordinate after-school activities and rides home.",
    "Phones can be used to document bullying or other inappropriate behavior in schools.",
    "Allowing phones in school teaches students responsible use of technology.",
    "Banning phones would reduce the risk of theft and property damage in schools.",
    "The radiation from multiple phones in a classroom could potentially be harmful to health.",
    "Phones can be disruptive when they ring or vibrate during class.",
    "Banning phones would make it harder for students to balance part-time jobs and school responsibilities.",
    "Phones can be used to take photos of notes and assignments, helping students stay organized.",
    "Allowing phones in school helps bridge the digital divide for students without internet access at home.",
    "The use of phones in school undermines the authority of teachers and school administration."
    ]
    '''
    opinions = [
        "Students need phones for academic work.",
        "Using phones help students because a lot of them dont have calculators so they use a calculator app",
        "Phones also allow students to offer video recordings of their work in class"

    ]
    '''    
    previous_results = {}

    for opinion in opinions:
        result = analyze_opinion(question, opinion, previous_results)
        if result:
            core_reason = result['core_reason']
            if core_reason not in previous_results:
                previous_results[core_reason] = {
                    'sentiment': result['sentiment'],
                    'nuances': [],
                    'count': 1
                }
            else:
                previous_results[core_reason]['count'] += 1
            
            if result['nuance']:  # Only add non-empty nuances
                previous_results[core_reason]['nuances'].append(result['nuance'])
            
            store_result(result, file_name)

    print("Core Reasons and Nuances:")
    for reason, data in previous_results.items():
        print(f"{reason} ({'pro' if data['sentiment'] else 'con'}) (mentioned {data['count']} times):")
        if data['nuances']:
            for nuance in data['nuances']:
                print(f"- {nuance}")
        else:
            print("- No specific nuances recorded.")
        print()
    

Core Reasons and Nuances:
Increased phone use harms mental health. (pro) (mentioned 4 times):
- Excessive phone use correlates with youth depression.
- Smartphones facilitate bullying beyond school hours.
- Radiation exposure from phones may affect students.

Phones hinder student interaction. (pro) (mentioned 8 times):
- Phones create barriers to face-to-face communication.
- Students prioritize phone use over engagement.
- Phone distractions lead to lower academic performance.
- Banning phones can reduce online harassment.
- Cheating undermines trust among students.
- Phones distract students from respecting authority.

Instant communication enhances safety. (con) (mentioned 4 times):
- Smartphones facilitate quick emergency responses.
- Phones help students manage health emergencies.
- Phones facilitate planning for group activities.
- Phones can provide evidence against bullying.

Phones enhance classroom learning. (con) (mentioned 5 times):
- Apps can foster innovative teaching me